## Import standard libraries

In [15]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
import alpaca_trade_api as tradeapi
import pandas as pd
import os as os
import tensorflow as tf
from tensorflow.keras import datasets, layers, models

# Set defaults for plotting
from IPython import display
%matplotlib inline
plt.rcParams['font.size'] = 15
plt.rcParams['figure.figsize'] = (7, 5)

import warnings
warnings.simplefilter('ignore')
#warnings.filterwarnings(action='once') # 'ignore'

## Import the data

In [45]:
# Define dates and time window
startDate = '2019-05-01'
endDate = '2020-05-28'
timeWindow = 20  # Number of time periods that will comprise the gramian angular field
timeIncrement = 'day'

X = []
for _, _, files in os.walk('./trainingData_Gramian/buy/'):
    Nbuy = len(files)
    for file in files:
        X.append(np.load('./trainingData_Gramian/buy/' + file))

for _, _, files in os.walk('./trainingData_Gramian/sell/'):
    Nsell = len(files)
    for file in files:
        X.append(np.load('./trainingData_Gramian/sell/' + file))

y = np.append(np.tile([1,0], (Nsell,1)), np.tile([0,1], (Nbuy,1)), axis=0)

## Separate data into test and train

In [54]:
trainingData = np.zeros(len(X), dtype=int)
trainingData[:int(0.6*len(X))] = 1  # 60% of the data will be training (~trainingData will be the testing data)
np.random.shuffle(trainingData)
trainingData = trainingData.astype(bool)
print(trainingData[:10])

X[trainingData]  # Won't work bc its a list

[ True  True False  True  True  True False False  True False]


array([ True,  True, False,  True,  True,  True, False, False,  True,
       False,  True,  True, False, False,  True, False,  True,  True,
        True, False, False, False, False, False, False, False, False,
       False,  True,  True, False,  True,  True, False,  True,  True,
        True, False,  True,  True,  True, False,  True,  True,  True,
        True, False,  True,  True, False, False,  True,  True, False,
       False, False,  True,  True, False,  True,  True,  True, False,
        True, False,  True,  True,  True,  True,  True,  True, False,
        True,  True,  True,  True,  True,  True,  True, False, False,
        True, False,  True, False, False, False, False,  True,  True,
        True,  True, False, False,  True, False,  True,  True,  True,
        True, False,  True,  True, False, False,  True,  True,  True,
        True,  True, False, False,  True,  True,  True,  True, False,
       False, False,  True, False,  True,  True,  True, False,  True,
        True,  True,

## Define the CNN model following the TF example

In [ ]:
model = models.Sequential()
model.add(layers.Conv2D(20, (3, 3), activation='relu', input_shape=(20, 20, 1)))  # arg2 is the kernel size
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.Flatten())
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(2))  # Only have two classes to predict: Buy or sell ([0,1] or [1,0])

## Compile the model using the training data

In [ ]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

history = model.fit(train_images, train_labels, epochs=10, validation_data=(test_images, test_labels))